# Gaussova eliminacija
---

## Općenito

Sustav $Ax=b$
se rješava u tri koraka (__bez pivotiranja__):

1. $A=LU$ (LU rastav, $O(\frac{2}{3}n^3)$ operacija),
2. $Ly=b$ (donje trokutrasti sustav, $n^2$ operacija),
3. $Ux=y$ (gornje torkutasti sustav, $n^2$ operacija).

Ukoliko pivotiramo, tada je

1. $PA=LU$, 
2. $Ly=P^T b$,
3. $Ux=y$. 

## LU rastav

In [1]:
function mylu{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    U=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n]) # This acccepts blocks and numbers
    L=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n])
    for k=1:n
        L[k,k]=one(A[1,1])
        for i=k+1:n
            L[i,k]=A[i,k]/A[k,k]
            for j=k+1:n
                A[i,j]=A[i,j]-L[i,k]*A[k,j]
            end
        end
        for j=k:n
            U[k,j]=A[k,j]
        end
    end
    L,U
end

mylu (generic function with 1 method)

In [2]:
A=rand(6,6)
# A=[2.0 2;3 4]

6×6 Array{Float64,2}:
 0.628172  0.936693  0.478618   0.251959   0.398775  0.69011 
 0.406099  0.264524  0.788246   0.98849    0.364594  0.348385
 0.775766  0.628616  0.214203   0.917419   0.515457  0.439628
 0.578453  0.92768   0.0401973  0.0758994  0.607237  0.117391
 0.948931  0.866254  0.169624   0.0889021  0.927775  0.631338
 0.44039   0.594936  0.29094    0.352142   0.487217  0.576982

In [3]:
L,U=mylu(A)

([1.0 0.0 … 0.0 0.0; 0.646477 1.0 … 0.0 0.0; … ; 1.51062 1.60907 … 1.0 0.0; 0.701066 0.181063 … 0.0226192 1.0], [0.628172 0.936693 … 0.398775 0.69011; 0.0 -0.341026 … 0.106796 -0.0977559; … ; 0.0 0.0 … 1.64093 -1.98848; 0.0 0.0 … 0.0 0.446735])

In [4]:
L*U-A

6×6 Array{Float64,2}:
 0.0  0.0   0.0           0.0           0.0           0.0        
 0.0  0.0   0.0           0.0           0.0           0.0        
 0.0  0.0  -2.22045e-16  -1.11022e-16   0.0           0.0        
 0.0  0.0   0.0           0.0           0.0          -5.55112e-17
 0.0  0.0  -8.32667e-17  -4.16334e-17   0.0           2.22045e-16
 0.0  0.0   0.0           0.0          -5.55112e-17   1.11022e-16

## Trokutasti sustavi

In [5]:
function myU{T}(U::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=n:-1:1
       for j=n:-1:i+1
            b[i]=b[i]-U[i,j]*b[j]
       end
        b[i]=b[i]/U[i,i]
    end
    b
end

function myL{T}(L::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=1:n
        for j=1:i-1
            b[i]=b[i]-L[i,j]*b[j]
        end
        b[i]=b[i]/L[i,i]
    end
    b
end

myL (generic function with 1 method)

In [6]:
b=rand(6)

6-element Array{Float64,1}:
 0.569711
 0.800761
 0.74422 
 0.836216
 0.385965
 0.415807

In [9]:
# Rijesimo susatav pomocu ugradjene funkcije:
x=A\b

6-element Array{Float64,1}:
 -0.368128
  0.967769
  0.357736
  0.582959
  0.305559
 -0.790444

In [10]:
# Rijesimo sustav pomocu nasih funkcija
y=myL(L,b)

6-element Array{Float64,1}:
  0.569711
  0.432456
 -0.629109
  0.568045
  2.07318 
 -0.353119

In [11]:
x1=myU(U,y)

6-element Array{Float64,1}:
 -0.368128
  0.967769
  0.357736
  0.582959
  0.305559
 -0.790444

In [12]:
# Usporedimo rjesenja
x-x1

6-element Array{Float64,1}:
  6.10623e-16
 -1.11022e-16
  0.0        
 -2.22045e-16
 -3.33067e-16
 -1.11022e-16

## Brzina

Program `mylu()` je jako spor. Između ostalog, alocira nepotrebno tri matrice i ne računa s blok matricama.

Program se može reformulirati na načun da su i $L$ i $U$ spremljene u polje $A$, pri čemu se dijagonala od $L$ ne sprema, jer su svi elementi jednaki 1 (vidi [Introduction to Linear Algebra, str. 100][St09]):

[St09]: https://books.google.hr/books?id=M19gPgAACAAJ&dq=strang%20introduction&hl=hr&source=gbs_book_other_versions "Gilbert Strang, 'Introduction to Linear Algebra, 4th Edition', Wellesley-Cambridge Press, 2009"


In [15]:
function mylu1{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        rho=k+1:n
        A[rho,k]=A[rho,k]/A[k,k]
        A[rho,rho]=A[rho,rho]-A[rho,k]*A[k,rho]'
    end
    A
end

mylu1 (generic function with 1 method)

In [16]:
mylu1(A)

6×6 Array{Float64,2}:
 0.628172   0.936693   0.478618   0.251959   0.398775    0.69011  
 0.646477  -0.341026   0.478831   0.825604   0.106796   -0.0977559
 1.23496    1.54874   -1.11845   -0.672383  -0.142411   -0.261231 
 0.920851  -0.190968   0.276362   0.187367   0.299777   -0.464571 
 1.51062    1.60907    1.18365   -4.39937    1.64093    -1.98848  
 0.701066   0.181063   0.117395   0.560132   0.0226192   0.446735 

In [17]:
L,U

([1.0 0.0 … 0.0 0.0; 0.646477 1.0 … 0.0 0.0; … ; 1.51062 1.60907 … 1.0 0.0; 0.701066 0.181063 … 0.0226192 1.0], [0.628172 0.936693 … 0.398775 0.69011; 0.0 -0.341026 … 0.106796 -0.0977559; … ; 0.0 0.0 … 1.64093 -1.98848; 0.0 0.0 … 0.0 0.446735])

Usporedimo brzine LAPACK-ovog programa `lu()` i našeg naivnog programa `mylu()`na većoj dimenziji. 

Izvedite program par puta radi točnijeg mjerenja brzine.

In [18]:
n=512
A=rand(n,n)

512×512 Array{Float64,2}:
 0.802976   0.347976    0.613713   …  0.906       0.800525   0.351915   
 0.484721   0.75696     0.810644      0.666111    0.590489   0.806365   
 0.690035   0.891139    0.119475      0.439374    0.65861    0.960242   
 0.0652302  0.241665    0.978775      0.221564    0.927762   0.130658   
 0.444875   0.275888    0.0458507     0.0878088   0.130668   0.000670261
 0.106079   0.976306    0.171811   …  0.83509     0.800158   0.33001    
 0.727205   0.431685    0.716797      0.00994195  0.846427   0.683664   
 0.644863   0.847278    0.169189      0.757541    0.75023    0.547744   
 0.649868   0.784643    0.603577      0.0169546   0.618999   0.587095   
 0.896996   0.53913     0.492618      0.95369     0.0548147  0.408084   
 0.5831     0.00961019  0.638611   …  0.0561616   0.817528   0.122279   
 0.686008   0.106396    0.426153      0.345416    0.659706   0.889769   
 0.309874   0.62674     0.478444      0.765782    0.131561   0.729443   
 ⋮                       

In [21]:
@time lu(A);

  0.078868 seconds (21 allocations: 6.009 MiB)


In [22]:
@time mylu1(A);

  0.803263 seconds (9.07 k allocations: 1.003 GiB, 18.86% gc time)


### Blok varijanta

`mylu()` je nekoliko desetaka puta sporiji od `lu()`. `mylu1()` je malo sporiji od `lu()`.

Probajmo s blokovima:

Preradimo `mylu1()` za rad s blokovima (nemamo ugrađeno pivotiranje!)

In [23]:
function mylu2{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        for rho=k+1:n
            A[rho,k]=A[rho,k]/A[k,k]
            for l=k+1:n
                A[rho,l]=A[rho,l]-A[rho,k]*A[k,l]
            end
        end
    end
    A
end

mylu2 (generic function with 1 method)

Napravimo prvo mali test:

In [24]:
# Probajte k,l=32,16 i k,l=64,8
k,l=4,4
Ab=[rand(k,k) for i=1:l, j=1:l];

In [25]:
A0=mylu2(Ab)

4×4 Array{Array{Float64,2},2}:
 [0.928297 0.6371 0.0106939 0.411772; 0.960074 0.172627 0.545185 0.660187; 0.920086 0.744216 0.796063 0.785019; 0.468725 0.233466 0.758991 0.0255357]       …  [0.876171 0.399069 0.43254 0.223368; 0.466518 0.607289 0.650238 0.692858; 0.175976 0.675274 0.3353 0.128007; 0.0768076 0.68086 0.6757 0.617048]              
 [0.0713834 0.264434 -0.0655305 0.944884; 0.448553 -1.05758 0.633758 0.726483; -0.173222 -0.703436 1.7098 -0.778318; -0.0195787 -0.300495 1.2683 0.142859]     [0.006575 -0.419311 -0.438419 -0.411462; 0.728361 0.536364 0.483295 0.894272; 1.04926 0.44976 1.36813 1.78363; 0.561859 -0.739016 0.101367 0.0946634]        
 [0.627248 -0.832953 0.72418 0.0918552; 0.192013 -0.925871 1.40663 -0.090687; 0.0457607 0.875003 0.212892 -0.253978; -1.00001 -0.255398 1.71691 -0.553985]     [-1.38135 -1.77644 -1.87972 -2.42792; 0.847216 -0.624806 0.185989 0.716069; -0.277571 0.543918 0.319629 0.125582; -0.403779 1.79695 0.167983 0.545717]       
 [-0.0242382 0.432

In [26]:
# Provjera
U=triu(A0)
L=tril(A0)
for i=1:maximum(size(L))
    L[i,i]=eye(L[1,1])
end

In [27]:
Res=L*U-Ab

4×4 Array{Array{Float64,2},2}:
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]                                                                                                 …  [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]                                                    
 [0.0 1.11022e-16 -1.11022e-16 5.55112e-17; 0.0 0.0 0.0 1.67401e-16; 1.66533e-16 0.0 -1.66533e-16 1.11022e-16; 0.0 0.0 0.0 2.22045e-16]                                  [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 -1.11022e-16 0.0; 0.0 0.0 0.0 0.0]                                           
 [-5.55112e-17 0.0 2.77556e-17 1.11022e-16; 0.0 -4.44089e-16 0.0 2.22045e-16; -1.11022e-16 -5.55112e-17 5.55112e-17 -1.11022e-16; -1.11022e-16 -3.33067e-16 0.0 0.0]     [0.0 -2.22045e-16 -2.22045e-16 -2.22045e-16; 1.11022e-16 -1.11022e-16 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 5.55112e-17]
 [0.0 1.11022e-16 -1.11022e-16 0.0; 4.16334e-17 -5.55112e-17 -5.55112e-17 2.22045e-16; 0.0 -1.11022

In [28]:
# pretvaranje blok matrice u obicnu
unblock(A) = mapreduce(identity, hcat, [mapreduce(identity, vcat, A[:,i]) for i = 1:size(A,2)])

unblock (generic function with 1 method)

In [29]:
norm(unblock(Res))

1.1145231103951312e-15

Sada probajmo veću dimenziju:

In [30]:
# Probajmo vece dimenzije (n=k*l)
k,l=32,16
Ab=[rand(k,k) for i=1:l, j=1:l];

In [31]:
@time mylu2(Ab);

  0.108365 seconds (3.95 k allocations: 26.553 MiB, 4.68% gc time)


Vidimo da je `mylu2()` gotovo jednako brz kao `lu()`, uz napomenu da `mylu2()` nema ugrađeno pivotiranje. 

## Pivotiranje

Stanardne implementacije uvijek računaju Gaussovu eliminaciju s _parcijalnim pivotiranjem_:

> u svakom koraku se retci pivotiranju tako da pivotni element ima najveću apsolutnu vrijednst u danom stupcu. Na taj 
> način je 
> 
> $$|L_{ij}| \leq 1,$$
> 
> što u praksi dovoljno spriječava rast elemenata.


In [32]:
A=rand(5,5)
L,U,P=lu(A)

([1.0 0.0 … 0.0 0.0; 0.882478 1.0 … 0.0 0.0; … ; 0.659314 0.010039 … 1.0 0.0; 0.523201 0.192506 … -0.0179526 1.0], [0.685735 0.166081 … 0.670698 0.436625; 0.0 0.67506 … -0.303792 0.346482; … ; 0.0 0.0 … -0.398994 -0.546478; 0.0 0.0 … 0.0 -0.446605], [5, 1, 3, 2, 4])

In [33]:
L*U-A[P,:]

5×5 Array{Float64,2}:
  0.0          0.0           0.0          0.0  0.0
  0.0          0.0           0.0          0.0  0.0
 -2.77556e-17  0.0           0.0          0.0  0.0
 -5.55112e-17  0.0           0.0          0.0  0.0
 -5.55112e-17  2.77556e-17  -5.55112e-17  0.0  0.0

### Potpuno pivotiranje

Sljedeći program računa Gaussovu eliminaciju s _potpunim pivotiranjem_ - u svakom koraku retci i stupci zamijene na način da se na pivotnu poziciju dovede element koji ima najveću apsolutnu vrijednost u trenutnoj podmatrici.

In [40]:
function gecp{T}(A1::Array{T})
    # Gaussova eliminacija s potpunim pivotiranjem
    # Izlaz: Pr*L*U*Pc'=A ili Pr'*A*Pc=L*U
    A=deepcopy(A1)
    n,m=size(A)
    Pr=eye(n,n)
    Pc=eye(n,n)
    D=zeros(n)
    for i=1:n-1
        am1,im1=findmax(abs.(A[i:n,i:n]),1)
        am,JJ=findmax(am1)
        II=mod(im1[JJ],(n-i+1))
        if II==0
            II=n-i+1
        end
        imax=II+i-1
        jmax=JJ+i-1
        #  zamijena redaka
        if (imax != i)
            temp = Pr[:,i]
            Pr[:,i] = Pr[:,imax]
            Pr[:,imax] = temp
            temp = A[i,:]
            A[i,:] = A[imax,:]
            A[imax,:] = temp
        end
        # zamijena stupaca
        if (jmax != i)
            temp = Pc[:,i]
            Pc[:,i] = Pc[:,jmax]
            Pc[:,jmax] = temp
            temp = A[:,i]
            A[:,i] = A[:,jmax]
            A[:,jmax] = temp
        end
        # eliminacija
        D[i]=A[i,i]
        A[i+1:n,i] = A[i+1:n,i]/D[i]
        A[i+1:n,i+1:n] = A[i+1:n,i+1:n] - A[i+1:n,i]*A[i,i+1:n]'
        A[i,i+1:n]=A[i,i+1:n]/D[i]
    end
    D[n]=A[n,n]
    L=eye(n,n)+tril(A,-1)
    U=eye(n,n)+triu(A,1)
    U=diagm(D)*U
    L,U,Pr,Pc
end

gecp (generic function with 1 method)

In [41]:
n=5
A=rand(n,n)
b=rand(n)

5-element Array{Float64,1}:
 0.153087
 0.134864
 0.559608
 0.165416
 0.346992

In [42]:
L,U,Pr,Pc=gecp(A)

([1.0 0.0 … 0.0 0.0; 0.967749 1.0 … 0.0 0.0; … ; 0.309465 -0.336513 … 1.0 0.0; 0.755108 0.870778 … -0.616338 1.0], [0.947858 0.888318 … 0.333876 0.19378; 0.0 -0.72934 … 0.578925 0.0652485; … ; 0.0 0.0 … 0.597666 0.332758; 0.0 0.0 … 0.0 -0.0551635], [0.0 0.0 … 0.0 1.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 1.0; … ; 0.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0])

In [43]:
Pr*L*U*Pc'-A

5×5 Array{Float64,2}:
  5.55112e-17   0.0          -1.11022e-16  0.0          0.0
  2.77556e-17   0.0           0.0          0.0          0.0
 -6.93889e-18  -5.55112e-17   0.0          0.0          0.0
  2.77556e-17   0.0           0.0          0.0          0.0
  0.0           0.0           0.0          2.77556e-17  0.0

In [44]:
y=myL(L,Pr'*b)

5-element Array{Float64,1}:
 0.134864 
 0.0349018
 0.196133 
 0.544047 
 0.284517 

In [45]:
z=myU(U,y)

5-element Array{Float64,1}:
 -3.4754 
  2.08528
  5.33587
  3.78191
 -5.1577 

In [46]:
x=Pc*z

5-element Array{Float64,1}:
  5.33587
 -5.1577 
  3.78191
  2.08528
 -3.4754 

In [47]:
A*x-b

5-element Array{Float64,1}:
  8.88178e-16
 -2.22045e-16
  0.0        
  4.44089e-16
  6.66134e-16

## Točnost

Neka je zadan sustav $Ax=b$, pri čemu je matrica $A$ regularna.

Da bi primijenili koncepte iz bilježnice [NA04_Pogreska_unatrag_i stabilni_algoritmi](NA04_Pogreska_unatrag_i_stabilni_algoritmi.ipynb), potrebno je:

1. napraviti teoriju smetnje za dani problem
2. analizirati pogreške algoritma (Gaussove eliminacije)

### Teorija smetnje

Neka je 

$$
(A+\delta A)\hat x=(b+\delta b)
$$

za neki $\hat x=x+\delta x$.

Želimo ocijeniti 

$$
\frac{\| \hat x - x \|}{\| x\|} \equiv \frac{\| \delta x\|}{\| x\|}.
$$

Uvedimo oznake (npr. prema [Matrix Computations, poglavlje 2.6.2][GVL13])

$$
\delta A=\varepsilon F, \quad \delta b=\varepsilon f, \qquad \hat x=x(\varepsilon),
$$
čime smo dobili jednodimenzionalni problem 

$$
(A+\varepsilon F)x(\varepsilon)=b+\varepsilon f.
$$

za neke (nepoznate) matricu $F$ i vektor $f$. 

Deriviranje po $\varepsilon$ daje

$$
Fx(\varepsilon)+(A+\varepsilon F)\dot x(\varepsilon)=f.
$$

Uvrštavanje $\varepsilon=0$ daje

$$
F x+A\dot x(0)=f,
$$

odnosno

$$
\dot x(0)=A^{-1}(f-Fx).
$$

Taylorov razvoj oko $x(0)$ glasi

$$
x(\varepsilon)=x(0)+\varepsilon \dot x(0) +O(\varepsilon^2),
$$

odnosno, uz zanemarivanje člana $O(\varepsilon^2)$,

$$
\hat x-x=\varepsilon A^{-1}(f-Fx)=A^{-1} (\varepsilon f + \varepsilon F x) = A^{-1} (\delta b + \delta A x).
$$

Svojstva norme povlače

$$
\| \hat x-x\|\leq \| A^{-1} \| (\| \delta b \|  + \| \delta A \| \cdot \|  x\| ).
$$

Konačno, zbog $\| b\| \leq \| A\| \| x\|$, imamo

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \| A\|  \cdot \| A^{-1} \| \bigg(\frac{\| \delta b \|}{\|b\|}  + \frac{\| \delta A \|}{ \|  A\|} \bigg). \tag{1}
$$

Broj 
$$
\kappa(A)\equiv \| A\|  \cdot \| A^{-1} \|
$$ 

je __uvjetovanost__ (__kondicija__)  matrice $A$ i kazuje nam 

> koliko se relativno uvećaju relativne promjene u polaznim podacima (matrici $A$ i vektoru $b$).

Pogledajmo primjer iz [Numeričke matematike, str. 42][RS04]:


[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, Osijek, 2004."

In [48]:
A= [0.234 0.458; 0.383 0.750]

2×2 Array{Float64,2}:
 0.234  0.458
 0.383  0.75 

In [49]:
b=[0.224;0.367]

2-element Array{Float64,1}:
 0.224
 0.367

In [50]:
x=A\b

2-element Array{Float64,1}:
 -1.0
  1.0

In [51]:
δb=[0.00009; 0.000005]
x1=A\(b+δb)

2-element Array{Float64,1}:
 -0.241744
  0.612791

In [52]:
cond(A), norm(δb)/norm(b), norm(x1-x)/norm(x)

(11322.197586092605, 0.0002096449170953002, 0.6020311134825742)

In [53]:
δA=[-0.001 0;0 0]
x2=(A+δA)\b

2-element Array{Float64,1}:
 0.129518
 0.423193

In [54]:
cond(A), norm(δA)/norm(A), norm(x2-x)/norm(x)

(11322.197586092605, 0.0010134105230118603, 0.896804787832142)

### Pogreška Gaussove eliminacije

Prema [Matrix Computations, poglavlje 3.3][GVL13], za izračunate faktore
$\hat L$ i $\hat U$ vrijedi

$$
\hat L\cdot \hat U = A+\delta A
$$

gdje je (nejednakost se čita po elementima matrica, $\varepsilon$ je sada točnost stroja)

$$
| \delta A|\leq 3(n-1) \varepsilon (|A|+|\hat L| \cdot |\hat U|) +O(\varepsilon^2).
$$

Zanemarivanje člana $O(\varepsilon^2)$ i prelazak na normu daju

$$
\|\delta A \| \approx \leq  O(n)\varepsilon (\| A\| + \| \hat L\| \cdot \| \hat U\|),
$$

pa je 

$$
 \frac{\|\delta A \|}{\|A\|} \leq O(n)\varepsilon \bigg(1+\frac{\| \hat L\| \cdot \| \hat U\|}{\|A\|}\bigg).
$$

Ukoliko se Gaussova eliminacija radi s pivotiranjem, tada će najvjerojatnije zadnji kvocijent također biti malen 
($\approx 1$). Također, pogreška kod rješavanja trokutastih sustava nije veća od navedene pa, uvrštavanjme u (1), slijedi 
da za relativnu pogrešku izračunatog rješenja vrijedi

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \kappa(A) O(n\varepsilon).
$$

> __Ukoliko je kondicija matrice velika, rješenje može biti netočno.__

[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

In [55]:
n=10
v=rand(n)

10-element Array{Float64,1}:
 0.550738
 0.978376
 0.102306
 0.810004
 0.110933
 0.915747
 0.686804
 0.440134
 0.59432 
 0.703396

In [57]:
# Vandermonmde-oove matrice imaju veliku kondiciju.
A=Array(Float64,n,n)
for i=1:n
    A[:,i]=v.^(i-1)
end
A=A'

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64, ::Int64) at .\deprecated.jl:57
 [3] include_string(::String, ::String) at .\loading.jl:515
 [4] include_string(::Module, ::String, ::String) at C:\Users\Ivan\.julia\v0.6\Compat\src\Compat.jl:407
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\Ivan\.julia\v0.6\IJulia\src\execute_request.jl:154
 [6] eventloop(::ZMQ.Socket) at C:\Users\Ivan\.julia\v0.6\IJulia\src\eventloop.jl:8
 [7] (::IJulia.##14#17)() at .\task.jl:335
while loading In[57], in expression starting on line 2


10×10 Array{Float64,2}:
 1.0         1.0       1.0          …  1.0          1.0         1.0      
 0.550738    0.978376  0.102306        0.440134     0.59432     0.703396 
 0.303312    0.95722   0.0104664       0.193718     0.353216    0.494766 
 0.167045    0.936522  0.00107077      0.0852621    0.209924    0.348016 
 0.0919981   0.916271  0.000109546     0.0375268    0.124762    0.244793 
 0.0506668   0.896458  1.12072e-5   …  0.0165168    0.0741484   0.172186 
 0.0279041   0.877073  1.14656e-6      0.00726963   0.0440679   0.121115 
 0.0153678   0.858108  1.17299e-7      0.00319961   0.0261904   0.0851919
 0.00846364  0.839553  1.20003e-8      0.00140826   0.0155655   0.0599237
 0.00466125  0.821399  1.2277e-9       0.000619824  0.00925089  0.04215  

In [58]:
b=rand(n)

10-element Array{Float64,1}:
 0.0627204
 0.843522 
 0.869844 
 0.329481 
 0.843062 
 0.0603482
 0.314869 
 0.180867 
 0.325915 
 0.705797 

In [59]:
x=A\b

10-element Array{Float64,1}:
     -9.90692e6
 -99941.2      
      1.12137e5
     -2.67507e6
     -1.25716e5
      5.03952e5
     -4.94368e7
 752831.0      
      1.77479e7
      4.31276e7

In [60]:
cond(A)

1.063738558418797e9

In [61]:
Ab=Array(BigFloat,n,n)
bb=Array(BigFloat,n)
for i=1:n
    for j=1:n
        Ab[i,j]=convert(BigFloat,A[i,j])
    end
    bb[i]=convert(BigFloat,b[i])
end
xb=Ab\bb

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{BigFloat}, ::Int64, ::Int64) at .\deprecated.jl:57
 [3] include_string(::String, ::String) at .\loading.jl:515
 [4] include_string(::Module, ::String, ::String) at C:\Users\Ivan\.julia\v0.6\Compat\src\Compat.jl:407
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\Ivan\.julia\v0.6\IJulia\src\execute_request.jl:154
 [6] eventloop(::ZMQ.Socket) at C:\Users\Ivan\.julia\v0.6\IJulia\src\eventloop.jl:8
 [7] (::IJulia.##14#17)() at .\task.jl:335
while loading In[61], in expression starting on line 1
Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{BigFloat}, ::Int64) at .\deprecated.jl:57
 [3] include_string(::String, ::String) at .\loading.jl:515
 [4] include_string(::Module, ::String, ::String) at C:\Users\Ivan\.julia\v0.6\Compat\src\Compat.jl:407
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\Ivan\.julia\v0.6\IJulia\src\execute_request.jl:154
 [6

10-element Array{BigFloat,1}:
 -9.906919699680192429565028948239296360567481735031523568551406110073146628858168e+06
 -9.994115024218548008136574780302635968252135850666314506064922619139755217154064e+04
  1.121365458431713245288078341594749721051785564635743084842376117403754874930266e+05
 -2.675065531058043300719704124265528297478353612435699745322771570938373299249514e+06
 -1.257156999784780305067251121302453961389497386311112464678536540510633371377129e+05
  5.039523035990319521220478115621803062353736041449685418935644255628888123593404e+05
 -4.943684960790880279833583537642937793244241143534765978688277028757285683811566e+07
  7.528309965288460335327947594786453669015209868054596897118514262869608233035092e+05
  1.774794487251061753950328553189023089170445470290110978886357752589956540409436e+07
  4.312762703310641523281089341848223703519377479411813598364471985933704712828216e+07

In [62]:
norm(xb-x)/norm(xb)

3.450120828199102017970343706917639984536169426883718217684612014810441078673437e-10

### Umjetno loša kondicija

In [63]:
A=[1 1; 1 2]
b=[1;3]
x=A\b
@show x,cond(A)
A1=[1e-4 1e-4;1 2]
b1=[1e-4;3]
x1=A1\b1
x,cond(A1),x-x1

(x, cond(A)) = ([-1.0, 2.0], 6.854101966249685)


([-1.0, 2.0], 50000.00017991671, [8.88178e-16, -4.44089e-16])

## Rezidual


Izračunato rješenje $\hat x$ sustava $Ax=b$ je točno rješenje nekog sličnog sustava (vidi [Afternotes on Numerical Analysis, str. 128][Ste96]):


$$ 
(A+\delta A)\hat x=b. \tag{1}
$$

__Rezidual__ (ili __ostatak__) definiramo kao 

$$
r=b-A\hat x.
$$

Tada je 

$$
0=b-(A+\delta A)\hat x=r- \delta A\hat x
$$

pa je 

$$ 
\| r\| \leq \| \delta A\hat x \| \leq \| \delta A\| \cdot \|\hat x \|,
$$

odnosno

$$
\frac{\|  \delta A\|}{\|A \|} \geq \frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$

Dakle,

> ako  _relativni rezidual_ 
>
> $$ \frac{r}{\| A\| \cdot \|\hat x \|}$$
> 
> ima veliku normu, tada __rješenje nije izračunato stabilno.__

S druge strane, ako relativni rezidual ima malu normu, tada je rješenje izračunato stabilno. Naime, za

$$
\delta A=\frac{r\hat x^T}{\|\hat x\|^2}
$$

vrijedi (1):

$$
b-(A+\delta A)\hat x=(b-A\hat x)-\delta A \hat x = r-\frac{r\hat x^T \hat x}{\|\hat x\|^2}
= r-\frac{r \|\hat x^T \hat x\|}{\|\hat x\|^2}=r-r=0.
$$

Također vrijedi

$$
\frac{\|  \delta A\|}{\|A \|}  \leq  \frac{\|r\|\|\hat x \|}{\| A\| \cdot \|\hat x \|^2}=
\frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$




[Ste96]: https://books.google.hr/books?id=w-2PWh01kWcC&printsec=frontcover&hl=hr#v=onepage&q&f=false    "G. W. Stewart, 'Afternotes on Numerical Analysis', SIAM, Philadelphia, 1996"

In [64]:
r=b-A*x

2-element Array{Float64,1}:
 0.0
 0.0

In [65]:
norm(r)/(norm(A)*norm(x))

0.0